# Würfel Orientierung ermitteln

In [1]:
import numpy as np
import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt

## Definition des KSYS
- x: blau
- y: rot
- z: weiß

## Testszenario
Es wird ein Testszenario definiert:
1. Der Würfel liegt mit der gelben Seite nach oben auf dem Tisch
2. Der Würfel wird gegriffen und in die Kamera gehalten, sodass die weiße Seite sichtbar ist
3. Der Würfel wird um 90° gedreht, sodass die blaue Seite sichtbar ist
4. Der Würfel wird um 180° gedreht, sodass die gegenüberliegende grüne Seite sichtbar ist

Theoretisch reicht bereits die erste 90° Drehung, um das KSys zu bestimmen.

### Szenario 1

In [2]:
image1 = {
    "parentcolor": "W",
    "matrix": "OBORWORWR"
}

# Erste rotation, um das globale KSYS
rotation1 = {
    "axis": "x",
    "steps": 1,
}

image2 = {
    "parentcolor": "B",
    "matrix": "YWWGBYBYW"
}

# Zweite rotation, um das globale KSYS
rotation2 = {
    "axis": "y",
    "steps": 2,
}

image3 = {
    "parentcolor": "G",
    "matrix": "BRRBGGYGG"
}

### Szenario 2

In [3]:
image1_2 = {
    "parentcolor": "O",
    "matrix": "GWBGOYROW"
}

# Erste rotation, um das globale KSYS
rotation1_2 = {
    "axis": "x",
    "steps": 1,
}

image2_2 = {
    "parentcolor": "G",
    "matrix": "YBBGGRGGR"
}

# Zweite rotation, um das globale KSYS
rotation2_2 = {
    "axis": "y",
    "steps": 2,
}

image3_2 = {
    "parentcolor": "B",
    "matrix": "WYWWBYYGB"
}

### Lösung
gilt für beide Szenarien

In [4]:
solution = "OORBWWORRGWBGOYROWYBBGGRGGRWOYWRRYBBBGYYBWWYWORGYYOGBO"

## Version 1
Aktuelle Orientierung des Würfels ermitteln

In [5]:
def get_axis_by_facecolor(image: dict):
    """Ordnet der Seite eine Achse zu"""
    if image["parentcolor"] == "W":
        return "z"
    elif image["parentcolor"] == "Y":
        return "-z"
    elif image["parentcolor"] == "R":
        return "x"
    elif image["parentcolor"] == "O":
        return "-x"
    elif image["parentcolor"] == "B":
        return "y"
    elif image["parentcolor"] == "G":
        return "-y"
    else:
        raise ValueError("Fehler in 'detect_axis_by_facecolor': Es wurde keine gültige Parentcolor erkannt")
    
def get_cube_orientation(image1: dict, image2: dict, rotation:dict):
    """Ermittelt die Gesamtorientierung aus den ersten zwei Bildern.
    ACHTUNG: Es wird immer angenommen, dass die erste Rotation um die globale x-Achse gemacht wurde"""

    if image1["axis"] == "z": # weiß links
        if image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,1] # x zeigt in globale -z-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [-1,0,0] # x zeigt in globale -x-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung

    elif image1["axis"] == "-z": # gelb links
        if image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,-1] # x zeigt in globale -z-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [-1,0,0] # x zeigt in globale -x-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung

    elif image1["axis"] == "x": # rot links
        if image2["axis"] == "z": # weiß oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # gelb oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,0,-1] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung

    elif image1["axis"] == "-x": # orange links
        if image2["axis"] == "z": # weiß oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # gelb oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,0,-1] # z zeigt in globale -z-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
    
    elif image1["axis"] == "y": # blau links
        if image2["axis"] == "z": # weiß oben
            x = [-1,0,0] # x zeigt in globale x-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # gelb oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [0,0,-1] # z zeigt in globale -z-Richtung
        elif image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [1,0,0] # z zeigt in globale x-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,-1] # x zeigt in globale -z-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [-1,0,0] # z zeigt in globale -y-Richtung

    elif image1["axis"] == "-y": # grün links
        if image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [-1,0,0] # z zeigt in globale -x-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,-1] # x zeigt in globale -z-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [1,0,0] # z zeigt in globale x-Richtung
        elif image2["axis"] == "z": # weiß oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # grün oben
            x = [-1,0,0] # x zeigt in globale -x-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [0,0,-1] # z zeigt in globale -y-Richtung
    
    return (x,y,z)

In [6]:
image1["axis"] = get_axis_by_facecolor(image1)
image2["axis"] = get_axis_by_facecolor(image2)

ksys = get_cube_orientation(image1, image2, rotation1)
print("x: ", ksys[0])
print("y: ", ksys[1])
print("z: ", ksys[2])

x:  [1, 0, 0]
y:  [0, 0, 1]
z:  [0, -1, 0]


Als erster Anlauf schon ganz gut, allerdings sind die Rotationen hart gecoded, die Funktion gilt also aktuell nur für den Fall, dass die erste Rotation um +90° in x-Richtung war. Das muss noch etwas flexibler werden...

## Version 2
Aktuelle Orientierung des Würfels für variable rotationen ermitteln

In [7]:
FACE_AXIS = {
    "R": "y", "O": "-y",
    "W": "z", "Y": "-z",
    "G": "x", "B": "-x",
}

GLOBAL_KSYS = {
    "x": np.array([1, 0, 0]),
    "y": np.array([0, 1, 0]),
    "z": np.array([0, 0, 1]),
}


def get_rotation_matrix(axis:str, steps:int)->np.ndarray:
    """generiert die Rotationsmatrix aus 90° Schritten um die gegebene Achse"""
    if axis[0] == "-":
        steps *= -1
        axis = axis[1]

    angle = steps * (np.pi / 2)
    c, s = np.cos(angle), np.sin(angle)
    if axis == "x":
        return np.array([[1,0,0],[0,c,-s],[0,s,c]])
    if axis == "y":
        return np.array([[c,0,s],[0,1,0],[-s,0,c]])
    if axis == "z":
        return np.array([[c,-s,0],[s,c,0],[0,0,1]])

def rotate_axis(axis:np.ndarray, Mrot:np.ndarray):
    return Mrot @ axis

def get_cube_ksys(image1:dict, image2:dict):
    cube_ksys = {}
    if image1["axis"][0] == "-":
        cube_ksys[image1["axis"][1]] = -image1["orientation"].astype(int)
        val1 = -image1["orientation"]
        ax_1 = image1["axis"][1]
    else:
        cube_ksys[image1["axis"]] = +image1["orientation"].astype(int) 
        val1 = +image1["orientation"]   
        ax_1 = image1["axis"]
    
    if image2["axis"][0] == "-":
        cube_ksys[image2["axis"][1]] = -image2["orientation"].astype(int)
        val2 = -image2["orientation"]
        ax_2 = image2["axis"][1]
    else:
        cube_ksys[image2["axis"]] = +image2["orientation"].astype(int)
        val2 = +image2["orientation"]
        ax_2 = image2["axis"]

    # letzte Achse über Kreuzprodukt berechnen - Reihenfolge bestimmt über Vorzeichen
    if (ax_1 == "x" and ax_2 == "y") or (ax_1 == "y" and ax_2 == "z") or (ax_1 == "z" and ax_2 == "x"):
        val3 = np.cross(val1,val2) 
    elif (ax_1 == "x" and ax_2 == "z") or (ax_1 == "y" and ax_2 == "x") or (ax_1 == "z" and ax_2 == "y"):
        val3 = np.cross(val2,val1)

    

    if cube_ksys.get("x") is None:
        cube_ksys["x"] = val3.astype(int)
    elif cube_ksys.get("y") is None:
        cube_ksys["y"] = val3.astype(int)
    elif cube_ksys.get("z") is None:
        cube_ksys["z"] = val3.astype(int)

    return cube_ksys


### Für Szenario 1 ausführen

In [8]:
# 1. Bild erkannte Achse mappen
image1["axis"] = FACE_AXIS.get(image1["parentcolor"])   # Achse holen (x, y oder z)
image1["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image1["axis"], ": ", image1["orientation"].astype(int))

print(rotation1["steps"]*90, "° Rotation um ", rotation1["axis"])

# 1. Rotation anwenden
Mrot = get_rotation_matrix(rotation1["axis"], rotation1["steps"])
image1["orientation"] = rotate_axis(image1["orientation"], Mrot)

print(image1["axis"], ": ", image1["orientation"].astype(int))

# 2. Bild auswerten
image2["axis"] = FACE_AXIS.get(image2["parentcolor"])   # Achse holen (x, y oder z)
image2["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image2["axis"], ": ", image2["orientation"].astype(int))

cube_ksys = get_cube_ksys(image1, image2)

print("\nCalculated Cube ksys rotation matrix")
for key in cube_ksys:
    print(key, ": ", cube_ksys[key].astype(int))

z :  [0 0 1]
90 ° Rotation um  x
z :  [ 0 -1  0]
-x :  [0 0 1]

Calculated Cube ksys rotation matrix
z :  [ 0 -1  0]
x :  [ 0  0 -1]
y :  [1 0 0]


### Für Szenario 2 Ausführen

In [9]:
# 1. Bild erkannte Achse mappen
image1_2["axis"] = FACE_AXIS.get(image1_2["parentcolor"])   # Achse holen (x, y oder z)
image1_2["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image1_2["axis"], ": ", image1_2["orientation"].astype(int))

print(rotation1_2["steps"]*90, "° Rotation um ", rotation1_2["axis"])

# 1. Rotation anwenden
Mrot = get_rotation_matrix(rotation1_2["axis"], rotation1_2["steps"])
image1_2["orientation"] = rotate_axis(image1_2["orientation"], Mrot)

print(image1_2["axis"], ": ", image1_2["orientation"].astype(int))

# 2. Bild auswerten
image2_2["axis"] = FACE_AXIS.get(image2_2["parentcolor"])   # Achse holen (x, y oder z)
image2_2["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image2_2["axis"], ": ", image2_2["orientation"].astype(int))

cube_ksys = get_cube_ksys(image1_2, image2_2)

print("\nCalculated Cube ksys rotation matrix")
for key in cube_ksys:
    print(key, ": ", cube_ksys[key].astype(int))

-y :  [0 0 1]
90 ° Rotation um  x
-y :  [ 0 -1  0]
x :  [0 0 1]

Calculated Cube ksys rotation matrix
y :  [0 1 0]
x :  [0 0 1]
z :  [-1  0  0]


## Flächen umwandeln

- Wenn weiß oben:   Betrachtung der x-y-Ebene → 1: (x=1,y=1), 2: (x=1,y=2), 3: (x=1,y=2), 4: (x=2,y=1), ...
- Wenn grün oben:   Betrachtung der z-y-Ebene → 1: (y=1,z=3), 2: (y=2,z=3), 3: (y=3,z=3), 4: (y=1,z=2), ...
- Wenn orange oben: Betrachtung der z-x-Ebene → 1: (x=1,z=3), 2: (x=2,z=3), 3: (x=3,z=3), 4: (x=1,z=2), ...
- Wenn rot oben:    Betrachtung der z-x-Ebene → 1: (x=3,z=3), 2: (x=2,z=3), 3: (x=1,z=3), 4: (x=3,z=2), ...
- Wenn blau oben:   Betrachtung der z-y-Ebene → 1: (y=3,z=3), 2: (y=2,z=3), 3: (y=1,z=3), 4: (y=3,z=2), ...
- Wenn gelb oben:   Betrachtung der x-y-Ebene → 1: (x=3,y=1), 2: (x=3,y=2), 3: (x=3,y=3), 4: (x=2,y=1), ...

Ziel muss es sein, die Reihenfolge der erhaltenen Farben auf die korrekte Reihenfolge zu mappen

In [10]:
print("Aktuelle Flächennormale: ", image2_2["axis"])
print(
    "Rotation des Würfels um globales KSYS:",
    "\n\tx = ", cube_ksys["x"].astype(int),
    "\n\ty = ", cube_ksys["y"].astype(int),
    "\n\tz = ", cube_ksys["z"].astype(int)
)

Aktuelle Flächennormale:  x
Rotation des Würfels um globales KSYS: 
	x =  [0 0 1] 
	y =  [0 1 0] 
	z =  [-1  0  0]


An diesem Beispiel sieht man:
- x zeigt nach oben, d.h. es wird die y-z-Achse betrachtet.
- z zeigt in diesem Beispiel nach -x, was hier korrekt ist
- Es muss nichts gemapped werden

Als nächstes kann die zuerst betrachtete Fläche gemapped werden
- zuerst muss die Rotation wieder rückgängig gemacht werden

In [11]:
backwards_rotation1 = {
    "axis": rotation1["axis"],
    "steps": -rotation1["steps"]
}
Mrot = get_rotation_matrix(backwards_rotation1["axis"], backwards_rotation1["steps"])

previous_cube_x = rotate_axis(cube_ksys["x"], Mrot).astype(int)
previous_cube_y = rotate_axis(cube_ksys["y"], Mrot).astype(int)
previous_cube_z = rotate_axis(cube_ksys["z"], Mrot).astype(int)

print("Aktuelle Flächennormale: ", image1_2["axis"])
print(
    "Rotation des Würfels um globales KSYS (eigener Ansatz):",
    "\n\tx = ", previous_cube_x,
    "\n\ty = ", previous_cube_y,
    "\n\tz = ", previous_cube_z
)

Aktuelle Flächennormale:  -y
Rotation des Würfels um globales KSYS (eigener Ansatz): 
	x =  [0 1 0] 
	y =  [ 0  0 -1] 
	z =  [-1  0  0]


In [12]:
def rotate_cube_ksys(cube_ksys:dict, rotation: dict):
    x = cube_ksys["x"]
    y = cube_ksys["y"]
    z = cube_ksys["z"]
    
    M_rot = get_rotation_matrix(rotation["axis"], rotation["steps"])

    x_new = rotate_axis(x, M_rot).astype(int)
    y_new = rotate_axis(y, M_rot).astype(int)
    z_new = rotate_axis(z, M_rot).astype(int)

    cube_ksys_new = {
        "x": x_new,
        "y": y_new,
        "z": z_new
    }

    return cube_ksys_new

Das heißt, um die Matrix zu korrigieren, muss um 90° um die globale z-Achse gedreht werden

In [13]:
def get_corrected_color_matrix(cube_ksys:dict, image:dict):

    color_string = image["matrix"]
    color_matrix = np.array(list(color_string)).reshape((3, 3))

    print("Image axis: ", image["axis"])

    if image["axis"] == "x" or image["axis"] == "-x":
        # so weit drehen, dass z = [-1,0,0]
        z = cube_ksys["z"]
        if (z == [-1,0,0]).all():
            print("Keine Rotation nötig")
            corrected_matrix = color_matrix
        elif (z == [0,1,0]).all():
            # eine Drehung um 90° nach links nötig
            corrected_matrix = np.rot90(color_matrix, k=1)
        elif (z == [1,0,0]).all():
            # zwei Drehungen um 90° nach links nötig
            corrected_matrix = np.rot90(color_matrix, k=2)
        elif (z == [0,-1,0]).all():
            # drei Drehungen um 90° nach links nötig
            corrected_matrix = np.rot90(color_matrix, k=3)


    elif image["axis"] == "y" or image["axis"] == "-y":
        # so weit drehen, dass z = [-1,0,0]
        z = cube_ksys["z"]
        if (z == [-1,0,0]).all():
            # passt schon, keine Drehung nötig
            print("Keine Rotation nötig")
            corrected_matrix = color_matrix
        elif (z == [0,1,0]).all():
            # eine Drehung um 90° nach links nötig
            corrected_matrix = np.rot90(color_matrix, k=1)
        elif (z == [1,0,0]).all():
            # zwei Drehungen um 90° nach links nötig
            corrected_matrix = np.rot90(color_matrix, k=2)
        elif (z == [0,-1,0]).all():
            # eine Drehung um 90° nach rechts nötig
            corrected_matrix = np.rot90(color_matrix, k=3)


    elif image["axis"] == "z" or image["axis"] == "-z":
        # so weit drehen, dass y = [0,1,0]
        y = cube_ksys["y"]
        if (y == [0,1,0]).all():
            # passt schon
            print("Keine Rotation nötig")
            corrected_matrix = color_matrix
        elif (y == [-1,0,0]).all():
            # eine Drehung um 90° nach rechts nötig
            corrected_matrix = np.rot90(color_matrix, k=3)
        elif (y == [1,0,0]).all():
            # eine Drehung um 90° nach links nötig
            corrected_matrix = np.rot90(color_matrix, k=1)
        elif (y == [0,-1,0]).all():
            # zwei Drehungen um 90° nach links nötig
            corrected_matrix = np.rot90(color_matrix, k=2)

    corrected_color_string = ''.join(corrected_matrix.flatten())
    return corrected_color_string

## Finaler Test!
### Jetzt ein vollständiges Szenario aufzeichnen...

In [14]:
image1 = {
    "parentcolor": "W",
    "matrix": "RROWWBROO"
}

rotation1 = {
    "axis": "x",
    "steps": 1
}

image2 = {
    "parentcolor": "O",
    "matrix": "BYWWOOGGR"
}

rotation2 = {
    "axis": "y",
    "steps": 2
}

image3 = {
    "parentcolor": "R",
    "matrix": "YRBORBWWY"    
}

rotation3 = {
    "axis": "x",
    "steps": 1
}

image4 = {
    "parentcolor": "Y",
    "matrix": "OBGOYYGRO"    
}

rotation4 = {
    "axis": "y",
    "steps": 1
}

image5 = {
    "parentcolor": "B",
    "matrix": "BGYYBWWYW"    
}

rotation5 = {
    "axis": "y",
    "steps": 2
}

image6 = {
    "parentcolor": "G",
    "matrix": "RGGRGGBBY"    
}

Jetzt Schritt für Schritt durchgehen...
### Schritt 1: Bild 1 auswerten

In [15]:
image1["axis"] = FACE_AXIS.get(image1["parentcolor"])
image1["orientation"] = GLOBAL_KSYS.get("z")
print(image1["axis"], ": ", image1["orientation"].astype(int))

z :  [0 0 1]


### Schritt 2: Rotation anwenden

In [16]:
M_rot_1 = get_rotation_matrix(rotation1["axis"], rotation1["steps"])
image1["orientation"] = rotate_axis(image1["orientation"], M_rot_1)
print(image1["axis"], ": ", image1["orientation"].astype(int))

z :  [ 0 -1  0]


### Schritt 3: Bild 2 auswerten

In [17]:
image2["axis"] = FACE_AXIS.get(image2["parentcolor"])   # Achse holen (x, y oder z)
image2["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image2["axis"], ": ", image2["orientation"].astype(int))

-y :  [0 0 1]


### Schritt 4: Koordinatensystem des Würfels ermitteln

In [18]:
cube_ksys = get_cube_ksys(image1, image2)
print("\nCalculated Cube ksys rotation matrix")
for key in cube_ksys:
    print(key, ": ", cube_ksys[key].astype(int))


Calculated Cube ksys rotation matrix
z :  [ 0 -1  0]
y :  [ 0  0 -1]
x :  [-1  0  0]


### Schritt 5: Farbmatrix aus Bild 1 korriegieren

1. Würfel zurück rotieren

In [19]:
backwards_rotation1 = {
    "axis": rotation1["axis"],
    "steps": -rotation1["steps"]
}
M_rot_1_inverted = get_rotation_matrix(backwards_rotation1["axis"], backwards_rotation1["steps"])

previous_cube_x = rotate_axis(cube_ksys["x"], M_rot_1_inverted).astype(int)
previous_cube_y = rotate_axis(cube_ksys["y"], M_rot_1_inverted).astype(int)
previous_cube_z = rotate_axis(cube_ksys["z"], M_rot_1_inverted).astype(int)

print("Aktuelle Flächennormale: ", image1["axis"])
print(
    "Rotation des Würfels um globales KSYS:",
    "\n\tx = ", previous_cube_x,
    "\n\ty = ", previous_cube_y,
    "\n\tz = ", previous_cube_z
)

previous_cube_ksys = {
    "x": previous_cube_x,
    "y": previous_cube_y,
    "z": previous_cube_z
}

Aktuelle Flächennormale:  z
Rotation des Würfels um globales KSYS: 
	x =  [-1  0  0] 
	y =  [ 0 -1  0] 
	z =  [0 0 1]


2. Farbmatrix korriegieren

In [20]:
corrected_color_matrix_1 = get_corrected_color_matrix(previous_cube_ksys, image1)
image1["matrix"] = corrected_color_matrix_1
print("Color matrix for ", image1["parentcolor"], "-layer: ", image1["matrix"])

Image axis:  z
Color matrix for  W -layer:  OORBWWORR


### Schritt 6: Farbmatrix für Bild 2 korrigieren

In [21]:
corrected_color_matrix_2 = get_corrected_color_matrix(cube_ksys, image2)
image2["matrix"] = corrected_color_matrix_2
print("Color matrix for ", image2["parentcolor"], "-layer: ", image2["matrix"])

Image axis:  -y
Color matrix for  O -layer:  GWBGOYROW


### Schritt 7: nächste Rotation anwenden

In [22]:
cube_ksys = rotate_cube_ksys(cube_ksys, rotation2)
print(
    "Rotation des Würfels um globales KSYS:",
    "\n\tx = ", cube_ksys["x"],
    "\n\ty = ", cube_ksys["y"],
    "\n\tz = ", cube_ksys["z"]
)

Rotation des Würfels um globales KSYS: 
	x =  [1 0 0] 
	y =  [0 0 1] 
	z =  [ 0 -1  0]


### Schritt 8: nächstes Bild auswerten

In [23]:
image3["axis"] = FACE_AXIS.get(image3["parentcolor"])
corrected_color_matrix_3 = get_corrected_color_matrix(cube_ksys, image3)
image3["matrix"] = corrected_color_matrix_3
print("Color matrix for ", image3["parentcolor"], "-layer: ", image3["matrix"])

Image axis:  y
Color matrix for  R -layer:  WOYWRRYBB


### Schritte 7 & 8 wiederholen

In [24]:
cube_ksys = rotate_cube_ksys(cube_ksys, rotation3)
print(
    "Rotation des Würfels um globales KSYS (eigener Ansatz):",
    "\n\tx = ", cube_ksys["x"],
    "\n\ty = ", cube_ksys["y"],
    "\n\tz = ", cube_ksys["z"]
)

image4["axis"] = FACE_AXIS.get(image4["parentcolor"])
corrected_color_matrix_4 = get_corrected_color_matrix(cube_ksys, image4)
image4["matrix"] = corrected_color_matrix_4
print("Color matrix for ", image4["parentcolor"], "-layer: ", image4["matrix"])

Rotation des Würfels um globales KSYS (eigener Ansatz): 
	x =  [1 0 0] 
	y =  [ 0 -1  0] 
	z =  [ 0  0 -1]
Image axis:  -z
Color matrix for  Y -layer:  ORGYYOGBO


In [25]:
cube_ksys = rotate_cube_ksys(cube_ksys, rotation4)
print(
    "Rotation des Würfels um globales KSYS (eigener Ansatz):",
    "\n\tx = ", cube_ksys["x"],
    "\n\ty = ", cube_ksys["y"],
    "\n\tz = ", cube_ksys["z"]
)

image5["axis"] = FACE_AXIS.get(image5["parentcolor"])
corrected_color_matrix_5 = get_corrected_color_matrix(cube_ksys, image5)
image5["matrix"] = corrected_color_matrix_5
print("Color matrix for ", image5["parentcolor"], "-layer: ", image5["matrix"])

Rotation des Würfels um globales KSYS (eigener Ansatz): 
	x =  [ 0  0 -1] 
	y =  [ 0 -1  0] 
	z =  [-1  0  0]
Image axis:  -x
Keine Rotation nötig
Color matrix for  B -layer:  BGYYBWWYW


In [26]:
cube_ksys = rotate_cube_ksys(cube_ksys, rotation5)
print(
    "Rotation des Würfels um globales KSYS (eigener Ansatz):",
    "\n\tx = ", cube_ksys["x"],
    "\n\ty = ", cube_ksys["y"],
    "\n\tz = ", cube_ksys["z"]
)

image6["axis"] = FACE_AXIS.get(image6["parentcolor"])
corrected_color_matrix_6 = get_corrected_color_matrix(cube_ksys, image6)
image6["matrix"] = corrected_color_matrix_6
print("Color matrix for ", image6["parentcolor"], "-layer: ", image6["matrix"])

Rotation des Würfels um globales KSYS (eigener Ansatz): 
	x =  [0 0 1] 
	y =  [ 0 -1  0] 
	z =  [1 0 0]
Image axis:  x
Color matrix for  G -layer:  YBBGGRGGR


### Finale Überprüfung

In [27]:
print("Musterlösung: ", 
    solution[0:9], 
    solution[9:18], 
    solution[18:27], 
    solution[27:36], 
    solution[36:45], 
    solution[45:54]
)
print("Fläche 1: \t", image1["parentcolor"], " ", image1["matrix"])
print("Fläche 2: \t", image2["parentcolor"], " ", image2["matrix"])
print("Fläche 3: \t", image3["parentcolor"], " ", image3["matrix"])
print("Fläche 4: \t", image4["parentcolor"], " ", image4["matrix"])
print("Fläche 5: \t", image5["parentcolor"], " ", image5["matrix"])
print("Fläche 6: \t", image6["parentcolor"], " ", image6["matrix"])

Musterlösung:  OORBWWORR GWBGOYROW YBBGGRGGR WOYWRRYBB BGYYBWWYW ORGYYOGBO
Fläche 1: 	 W   OORBWWORR
Fläche 2: 	 O   GWBGOYROW
Fläche 3: 	 R   WOYWRRYBB
Fläche 4: 	 Y   ORGYYOGBO
Fläche 5: 	 B   BGYYBWWYW
Fläche 6: 	 G   YBBGGRGGR
